# Example of global pipeline using XTRAINS dataset

Using a yoloV8 model that has been trained to recognize types of wagon on images. 

Link to the XTRAINS dataset: [https://bitbucket.org/xtrains/dataset/src/master/](https://bitbucket.org/xtrains/dataset/src/master/)

In [1]:
from ultralytics import YOLO
import owlready2

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
from ontoclassifier import *

## Loading ontology

In [2]:
onto_filename = "data/xtrains_ontology.owl"
xtrains = owlready2.get_ontology(onto_filename).load()
owlready2.sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /Users/lewandowski/opt/anaconda3/envs/m1_pytorch_latest/lib/python3.11/site-packages/owlready2/hermit:/Users/lewandowski/opt/anaconda3/envs/m1_pytorch_latest/lib/python3.11/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/8b/62nv382d15q7777wx0q16s4w0000gn/T/tmphl4p1gfu
* Owlready2 * HermiT took 0.5347609519958496 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [3]:
print(list(xtrains.Train.descendants()))

[xtrains_ontology.RuralTrain, xtrains_ontology.WarTrain, xtrains_ontology.Train, xtrains_ontology.TypeC, xtrains_ontology.PassengerTrain, xtrains_ontology.TypeB, xtrains_ontology.FreightTrain, xtrains_ontology.LongTrain, xtrains_ontology.TypeA, xtrains_ontology.MixedTrain, xtrains_ontology.EmptyTrain, xtrains_ontology.LongFreightTrain]


## Global pipeline with Yolov8 Wrapper

In [4]:
# target_classes = [
#     xtrains.LongTrain,
#     xtrains.EmptyTrain,
#     xtrains.FreightTrain,
#     xtrains.PassengerTrain,
# ] # or use all descendants of Train: 
target_classes = xtrains.Train.descendants()

has_feature = OntoFeature(xtrains.has)
has_property_wrapper = YoloV8PropertyWrapper(
    YOLO("data/xtrains-yolov8.pt"),
    has_feature,
    {   # map yolo detected classes to ontology classes 
        0: ["Locomotive"],
        1: ["EmptyWagon", "LongWagon", "ReinforcedCar"],
        2: ["EmptyWagon", "LongWagon"],
        3: ["EmptyWagon", "ReinforcedCar"],
        4: ["EmptyWagon"],
        5: ["PassengerCar", "LongWagon", "ReinforcedCar"],
        6: ["PassengerCar", "LongWagon"],
        7: ["PassengerCar", "ReinforcedCar"],
        8: ["PassengerCar"],
        9: ["FreightWagon", "LongWagon", "ReinforcedCar"],
        10: ["FreightWagon", "LongWagon"],
        11: ["FreightWagon", "ReinforcedCar"],
        12: ["FreightWagon"]
    },
)
fe_yolo = Yolov8FeaturesExtractor([has_property_wrapper])

ontological_extractor = OntologicalExtractor({0: [fe_yolo]})

onto_classifier = OntoClassifier(xtrains, target_classes, ontological_extractor)

pipeline = torch.nn.Sequential(
        ontological_extractor,
        onto_classifier
)

explainer = OntologicalExtractorExplainer(ontological_extractor, onto_classifier)


In [5]:
onto_classifier.getTargettedClassesEncoder().classes_

array(['http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#EmptyTrain', 'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#FreightTrain', 'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#LongFreightTrain',
       'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#LongTrain', 'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#MixedTrain', 'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#PassengerTrain',
       'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#RuralTrain', 'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#Train', 'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#TypeA', 'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#TypeB',
       'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#TypeC', 'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#WarTrain'], dtype=object)

## Inference

In [6]:
from torchvision.io import read_image, ImageReadMode

images = ['data/xtrains-0082.png', 'data/xtrains-0123.png']

img_tensors = []
for img in images:
    img_tensors.append(read_image(img, ImageReadMode.RGB).float())

inputs = torch.stack(img_tensors)

In [7]:
results = pipeline(inputs)
pprint(onto_classifier.getTargettedClassesEncoder().inverse_transform(results))

[('http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#FreightTrain',
  'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#LongFreightTrain',
  'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#LongTrain',
  'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#Train',
  'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#TypeB'),
 ('http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#LongTrain',
  'http://www.semanticweb.org/lewandowski/ontologies/2023/2/xtrains#Train')]


## Explain results

In [8]:
one_input = torch.stack([img_tensors[0]])

explainer.explain(
    xtrains.LongFreightTrain, 
    one_input
    )


This is a LongFreightTrain because : 
  This is a Train because : 
    has SOME Locomotive or Wagon is True 
      found EmptyWagon, FreightWagon, [FreightWagon & LongWagon], Locomotive
  This is a FreightTrain because : 
    This is a Train because : 
      has SOME Locomotive or Wagon is True 
        found EmptyWagon, FreightWagon, [FreightWagon & LongWagon], Locomotive
    has MIN 2 FreightWagon is True 
      found 2 FreightWagon
  This is a LongTrain because : 
    has MIN 3 Wagon is True 
      found EmptyWagon, FreightWagon, [FreightWagon & LongWagon]



In [9]:
one_input = torch.stack([img_tensors[1]])

explainer = OntologicalExtractorExplainer(ontological_extractor, onto_classifier)

explainer.explain(
    xtrains.EmptyTrain, 
    one_input
    )


This is NOT a EmptyTrain because : 
  has ONLY EmptyWagon or Locomotive is False 
    found [FreightWagon & LongWagon]

